## 六，qr&MM

### *(一)分位数回归*

![1555133271812](https://github.com/HenaChris/-/blob/master/qr1.jpg?raw=true)

![qr2](https://github.com/HenaChris/-/blob/master/qr2.jpg?raw=true)



**定义分位数回归：**
<center>$Q_y(τ|x) = α_0 + α_1*x_1+α_2*x_2+....+α_n*x_n$


**分位数岭回归的目标函数为**：
<center>$min_β \frac{1}{n} \sum_{i=1}^{n}ρ_τ(y_i - x_i^{T}β)+λ_n‖ β ‖_2^{2}$

**中位数回归的目标函数为：**
<center>$min_β\sum_{i=1}^{n}|y_i-x_i^β|$

不能直接求导数计算，因此要引入MM算法进行计算。



### *(二) MM算法*

MM算法的主要思想：
找到一个辅助函数$Q(θ|θ^{t})$，去逼近似然函数$l(\theta|Y_{obs})$。步骤如下：

<center>令：$Q(θ|θ^{t})\leq l(\theta|Y_{obs})$,for any $\theta∈Θ$


<center>仅当$\theta=\theta^{(t)}$时，有$Q(θ_{(t)}|θ^{t})=l(\theta|Y_{obs})$

MM算法不直接最大化$l(\theta|Y_{obs})$，而是最大化$Q(θ|θ^{t})$：
<center>$θ^{(t+1)} = argmaxQ(\theta|\theta_{(t)})$

在这个过程中，我们最大化Q函数的同时，也在最大化似然函数：
<center>$l(\theta^{(t+1)}|Y_obs) \ge Q(\theta^{(t+1)}|\theta) \ge Q(\theta^{(t)}|\theta)=l(\theta^{(t)}|Y_{obs})$

### *（三）MM算法计算中位数回归*

1. 令$Q(β|β^{(l)}) = \sum_{i=1}^{n}\frac{(y_i-X_i^Tβ)^2}{2|y_i-X_i^Tβ^{(l)}|} + \frac{1}{2}|y_i-X_i^Tβ^{(l)}|$
    
2. 初始的β值为OLS估计

3. 给定$β^{(l)}$，可以计算$w_i=|y_i-x_i^Tβ^{(l)}|$

4. $W_{(l)}={diag(w_i^{(l)})}^{-1}$,对Q函数求导即可得之。
5. 重复以上步骤

In [1]:
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt
n=100

#data generation
x1=np.random.randn(n,1)
x2=np.random.randn(n,1)
X=np.hstack((x1,x2))
error = np.random.randn(n,1)*0.2
y =3*x1+5*x2+error

#中位数回归
def reg_llw(x,y,tol):
    beta1 = la.inv(X.T.dot(X)).dot(X.T).dot(y)#默认的β1
    diff = 1
    iter = 0
    path = []
    while np.abs(diff)>tol:#当目标函数差异小于容忍度时停止迭代
        iter = iter + 1
        L1sum0 = np.sum(np.abs(y-X.dot(beta1)))#目标函数
        w = np.abs(y - X.dot(beta1))
        W = np.diag(w.ravel()**(-1))
        beta1 = la.inv(X.T.dot(W).dot(X)).dot(X.T).dot(W).dot(y)#计算替代函数的最小值。
        L1sum1 = np.sum(np.abs(y - X.dot(beta1)))
        diff = L1sum1 - L1sum0#计算两次迭代的差异，差异小于容忍度时停止迭代
        path.append(L1sum0)#查看目标函数是否在减小
    return  beta1,iter,path
(beta1,iter,path) = reg_llw(X,y,0.00005)
print(beta1)

[[2.99323549]
 [4.97875694]]
